In [ ]:
#!pip install google-cloud-vision

import google.cloud.vision
print("Google Cloud Vision API is installed and ready!")

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "newhaiku.json" ## u need to create ur own authentication key and link it here
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

from google.cloud import vision
client = vision.ImageAnnotatorClient()
print("Vision API client successfully initialized!")

from google.cloud import aiplatform

def analyze_image(image_path):
    """Analyzes an image for object labels and facial emotions using Google Cloud Vision API."""

    client = vision.ImageAnnotatorClient()

    # Read image file
    with open(image_path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # Object detection
    label_response = client.label_detection(image=image)
    labels = label_response.label_annotations

    print("\n🎯 Detected Labels:")
    for label in labels:
        print(f"- {label.description} (Confidence: {label.score:.2f})")

    # Face emotion detection
    face_response = client.face_detection(image=image)
    faces = face_response.face_annotations

    if not faces:
        print("\n😊 No faces detected.")
    else:
        print("\n😊 Facial Emotion Analysis:")
        for i, face in enumerate(faces):
            print(f"\n👤 Face {i+1}:")
            print(f"   - Joy: {face.joy_likelihood}")
            print(f"   - Sorrow: {face.sorrow_likelihood}")
            print(f"   - Anger: {face.anger_likelihood}")
            print(f"   - Surprise: {face.surprise_likelihood}")

# get image
def analyze_image_labels(image_path):
    """Analyzes an image and returns a list of object labels."""
    client = vision.ImageAnnotatorClient()
    with open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    label_response = client.label_detection(image=image)
    labels = label_response.label_annotations
    label_list = [label.description for label in labels]
    return label_list


# Haiku Generation
def generate_haiku_ai(labels, project_id, location, endpoint_id):
    """Generates a haiku using a Google Cloud Vertex AI model."""
    aiplatform.init(project=project_id, location=location)
    endpoint = aiplatform.Endpoint(endpoint_name=f"projects/{project_id}/locations/{location}/endpoints/{endpoint_id}")
    prompt = f"Write a haiku about: {', '.join(labels)}. Be creative and follow the 5-7-5 syllable structure."
    instances = [{"prompt": prompt}]
    prediction = endpoint.predict(instances=instances)
    haiku = prediction.predictions[0]["content"]
    return haiku.strip()

# Example usage (replace with your values)
image_path = "ithaca.jpg"
project_id = "compact-marker-451921-b1"
location = "us-central1"
endpoint_id = "YOUR_ENDPOINT_ID" # Replace with your endpoint ID




# For Ithaca.jpg image
# Extract just the labels
labels = analyze_image_labels("ithaca.jpg")
print("\nExtracted Labels:", labels)

# Generate and print the haiku
try:
    haiku = generate_haiku_ai(labels, project_id, location, endpoint_id)
    print("\nGenerated Haiku (AI):")
    print(haiku)
except Exception as e:
    print(f"Error generating haiku: {e}")
    print("Please ensure you have set up Vertex AI correctly and have a model deployed.")

ModuleNotFoundError: No module named 'google.cloud.vision'

In [ ]:
import os
import vertexai
from google.cloud import vision
from vertexai.language_models import TextGenerationModel

# Set the path to your service account JSON file
SERVICE_ACCOUNT_FILE = "newhaiku.json"

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_FILE

# Project and Location
PROJECT_ID = "compact-marker-451921-b1"
LOCATION = "us-central1"

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Image Path
image_path = "ithaca.jpg"

# Vision AI Description
def describe_image(image_path):
    client = vision.ImageAnnotatorClient()
    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.annotate_image({
        'image': image,
        'features': [{'type_': vision.Feature.Type.LABEL_DETECTION}],
    })
    labels = [label.description for label in response.label_annotations]
    return ", ".join(labels)

image_description = describe_image(image_path)
print(f"Image Description: {image_description}")

# Text Bison Haiku Generation
def generate_haiku(image_description):
    model = TextGenerationModel.from_pretrained("text-bison")
    prompt = f"""
    Create a haiku based on the following image description:
    {image_description}
    Haiku:
    """
    response = model.predict(
        prompt,
        temperature=0.7,
        max_output_tokens=256,
        top_p=0.8,
        top_k=40,
    )
    return response.text

haiku = generate_haiku(image_description)
print(f"Generated Haiku:\n{haiku}")

Image Description: City, Residential area, Urban area, Neighbourhood, Landscape, Roof, Bird's-eye view, Human settlement, Urban design, Suburb


BadRequest: 400 POST https://us-central1-aiplatform.googleapis.com/v1/projects/compact-marker-451921-b1/locations/us-central1/publishers/google/models/text-bison@002:predict?%24alt=json%3Benum-encoding%3Dint: Project `965935594995` is not allowed to use Publisher Model `projects/compact-marker-451921-b1/locations/us-central1/publishers/google/models/text-bison@002`

In [ ]:
import os
import vertexai
from google.cloud import vision
from vertexai.language_models import TextGenerationModel

# Set the path to your service account JSON file
SERVICE_ACCOUNT_FILE = "newhaiku.json"

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_FILE

# Project and Location
PROJECT_ID = "compact-marker-451921-b1"
LOCATION = "us-central1"

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Image Path
image_path = "ithaca.jpg"

# Vision AI Description
def describe_image(image_path):
    client = vision.ImageAnnotatorClient()
    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.annotate_image({
        'image': image,
        'features': [{'type_': vision.Feature.Type.LABEL_DETECTION}],
    })
    labels = [label.description for label in response.label_annotations]
    return ", ".join(labels)

image_description = describe_image(image_path)
print(f"Image Description: {image_description}")

# Text Bison Haiku Generation
def generate_haiku(image_description):
    # Pass project to from_pretrained
    model = TextGenerationModel.from_pretrained("text-bison", project=PROJECT_ID)
    prompt = f"""
    Create a haiku based on the following image description:
    {image_description}
    Haiku:
    """
    response = model.predict(
        prompt,
        temperature=0.7,
        max_output_tokens=256,
        top_p=0.8,
        top_k=40,
    )
    return response.text

haiku = generate_haiku(image_description)
print(f"Generated Haiku:\n{haiku}")

Image Description: City, Residential area, Urban area, Neighbourhood, Landscape, Roof, Bird's-eye view, Human settlement, Urban design, Suburb


TypeError: _ModelGardenModel.from_pretrained() got an unexpected keyword argument 'project'